[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **Fine-Tune YOLO For Fruits Counting**

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)

## 🎯 Objective

This notebook demonstrates how to build an automated fruit counting system using a fine-tuned YOLO segmentation model. The project covers the entire workflow: extracting image frames from a source video, annotating the data, training a model to recognize and track fruits, and implementing a custom line-crossing logic to count them as they pass a specific point.


## 🚀 Key Features

* **Frame Extraction**: Automatically sample frames from a video to create an image dataset for training.
* **Format Conversion**: Convert annotations from COCO JSON to the YOLO segmentation format.
* **Model Training**: Fine-tune a pre-trained YOLOv11 segmentation model on the custom fruit dataset.
* **Object Tracking**: Use the BotSort tracker with the custom model to track individual fruits across frames.
* **Line-Crossing Counter**: Implement a custom logic to count objects as their tracked path intersects a predefined line.


## 📚 Libraries & Prerequisites

* **Core Libraries**: `ultralytics`, `opencv-python`, `matplotlib`, `numpy`.
* **Environment**: A Python environment with GPU support is highly recommended for training.
* **Dataset**: A source video of fruits (e.g., on a conveyor belt) and an `annotation.json` file for the extracted frames.


## Annotate your Custom dataset using Labellerr

 ***1. Visit the [Labellerr](https://www.labellerr.com/?utm_source=githubY&utm_medium=social&utm_campaign=github_clicks) website and click **“Sign Up”**.*** 

 ***2. After signing in, create your workspace by entering a unique name.***

 ***3. Navigate to your workspace’s API keys page (e.g., `https://<your-workspace>.labellerr.com/workspace/api-keys`) to generate your **API Key** and **API Secret**.***

 ***4. Store the credentials securely, and then use them to initialise the SDK or API client with `api_key`, `api_secret`.*** 


### Use Labellerr SDK for uploading and perform annotation of your own dataset

In [ ]:
# uncomment the following lines to install required packages in a Jupyter notebook environment

# !pip install git+https://github.com/Labellerr/SDKPython.git
# !pip install ipyfilechooser
# !git clone https://github.com/Labellerr/yolo_finetune_utils.git

In [ ]:
# all the imports required for this notebook
from labellerr.client import LabellerrClient
from labellerr.core.datasets import create_dataset_from_local
from labellerr.core.annotation_templates import create_template
from labellerr.core.projects import create_project
from labellerr.core.schemas import DatasetConfig, AnnotationQuestion, QuestionType, CreateTemplateParams, DatasetDataType, CreateProjectParams, RotationConfig
from labellerr.core.projects import LabellerrProject
from labellerr.core.exceptions import LabellerrError

import uuid
from ipyfilechooser import FileChooser

In [ ]:
api_key = input("YOUR_API_KEY")        # go to labellerr workspace to get your API key
api_secret = input("YOUR_API_SECRET")  # go to labellerr workspace to get your API secret
client_id = input("YOUR_CLIENT_ID")   # Contact labellerr support to get your client ID i.e. support@tensormatics.com

client = LabellerrClient(api_key, api_secret, client_id)


### ***STEP-1: Create a dataset on labellerr from your local folder***

The SDK supports in creating dataset by uploading local files. 

In [ ]:
# Create a folder chooser starting from a directory (for example, your home directory)
chooser = FileChooser('/')

# Set the chooser to folder selection mode only
chooser.title = 'Select a folder containing your dataset'
chooser.show_only_dirs = True

# Display the widget
display(chooser)

FileChooser(path='D:\', filename='', title='Select a folder containing your dataset', show_hidden=False, selec…

In [ ]:
path_to_dataset = chooser.selected_path
print("You selected:", path_to_dataset)

You selected: D:\Professional\Projects\Cell_Segmentation_using_YOLO\frames_output


In [ ]:
my_dataset_type = input("Enter your dataset type (video or image): ").lower()
print("Selected dataset type:", my_dataset_type)

Selected dataset type: video


In [ ]:
dataset = create_dataset_from_local(
    client=client,
    dataset_config=DatasetConfig(dataset_name="My Dataset", data_type="image"),
    folder_to_upload=path_to_dataset
)

print(f"Dataset created with ID: {dataset.dataset_id}")


### ***STEP-2: Create annotation project on labellerr of your created dataset***

Create a annotation project of your uploaded dataset to start performing annotation on labellerr UI

In [ ]:
# Create annotation guideline template for video annotation project (like classes to be annotated)

template = create_template(
    client=client,
    params=CreateTemplateParams(
        template_name="My Template",
        data_type=DatasetDataType.image,
        questions=[
            AnnotationQuestion(
                question_number=1,
                question="Object",
                question_id=str(uuid.uuid4()),
                question_type=QuestionType.polygon,
                required=True,
                color="#FF0000"
            )
        ]
    )
)
print(f"Annotation template created with ID: {template.annotation_template_id}")


In [ ]:
dataset.status()        # wait until dataset is processed before creating project

project = create_project(
    client=client,
    params=CreateProjectParams(
        project_name="My Project",
        data_type=DatasetDataType.image,
        rotations=RotationConfig(
            annotation_rotation_count=1,
            review_rotation_count=1,
            client_review_rotation_count=1
        )
    ),
    datasets=[dataset],
    annotation_template=template
)

print(f"✓ Project created: {project.project_id}")

Your project has been created now go to labellerr platform to perform annotation 

***click to go to labellerr.com***

[![Labellerr](https://cdn.labellerr.com/1%20%20Documentation/1c9dc7ce-9a54-4111-8fd5-0363ba3e00e1.webp)](https://www.labellerr.com/?utm_source=githubY&utm_medium=social&utm_campaign=github_clicks)
Open the project you created (Projects → select your project).

Click Start Labeling to open the annotation interface. Use the configured labeling tools (bounding boxes, polygon, dot, classification, etc.) to annotate files.
### ***STEP-3: Export your annotation in required format***

Generate a temporary download URL to retrieve your exported JSON file:

### Export Configuration Parameters

| Parameter | Type | Description |
|-----------|------|-------------|
| `export_name` | string | Display name for the export |
| `export_description` | string | Description of what this export contains |
| `export_format` | string | Output format (e.g., `json`, `xml`, `coco`) |
| `statuses` | list | Annotation statuses to include in export |

### Common Annotation Statuses

- **`review`**: Annotations pending review
- **`r_assigned`**: Review assigned to a reviewer
- **`client_review`**: Under client review
- **`cr_assigned`**: Client review assigned
- **`accepted`**: Annotations accepted and finalized

---

In [ ]:
export_config = {
    "export_name": "Weekly Export",
    "export_description": "Export of all accepted annotations",
    "export_format": "coco_json",
    "statuses": ['review', 'r_assigned','client_review', 'cr_assigned','accepted']
}

try:
    # Get project instance
    project = LabellerrProject(client=client, project_id=project.project_id)
    
    # Create export
    result = project.create_local_export(export_config)
    export_id = result["response"]['report_id']
    print(f"Local export created successfully. Export ID: {export_id}")
except LabellerrError as e:
    print(f"Local export creation failed: {str(e)}")
    
    
try:
    download_url = client.fetch_download_url(
        project_id=project.project_id,
        uuid=str(uuid.uuid4()),
        export_id=export_id
    )
    print(f"Download URL: {download_url}")
except LabellerrError as e:
    print(f"Failed to fetch download URL: {str(e)}")


Now you can download your annotations locally using given URL

## **Converting COCO-JSON to YOLO format**

The annotations from the previous step are in COCO JSON format, which needs to be converted into the YOLO segmentation format for training. We use a helper script from the cloned repository to perform this conversion automatically.

In [ ]:
from yolo_finetune_utils.coco_yolo_converter.seg_converter import coco_to_yolo_converter

coco_to_yolo_converter(json_path="annotation.json", images_dir="dataset", output_dir="yolo_format", seed=15)

### **Model Training**

With the dataset correctly formatted, we can now fine-tune the YOLO model. We'll use a pre-trained `yolo11m-seg.pt` model as a starting point and train it on our custom fruit dataset for 250 epochs.

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
!yolo task=segment mode=train data="./yolo_format/data.yaml" model="yolo11m-seg.pt" epochs=250 imgsz=640 batch=30

### **Tracking using Custom Model**

After training, we can use our custom model for tracking objects in a video. The `yolo` command provides a simple way to apply the model and a specified tracker (like `botsort.yaml`) to a video, saving the output with visual tracking information.

In [ ]:
!yolo task=segment mode=track tracker=botsort.yaml model="./runs/segment/train/weights/best.pt" conf=0.2 source="./assests/2.mp4" save=True show_labels=True

### **Drawing Counter Line**

To implement our custom counter, we first need to define a virtual line on the video frame. This section of code defines the line's coordinates based on the video's dimensions and then visualizes it on a sample frame to confirm its position.

In [ ]:
video_path = r'assests\2.mp4'  # ← VIDEO PATH

In [ ]:
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"❌ Error: Cannot open video file: {video_path}")

cap.set(cv2.CAP_PROP_POS_FRAMES, 100)  # Set to frame number 100
ret, frame = cap.read()
cap.release()

plt.figure(figsize=(10, 6))
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [ ]:
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"❌ Error: Cannot open video file: {video_path}")
else:
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
print(f"Width: {width}, Height: {height}")

In [ ]:
line = (1500,0), (1500,1080)

In [ ]:
start_point, end_point = line

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"❌ Error: Cannot open video file: {video_path}")

cap.set(cv2.CAP_PROP_POS_FRAMES, 100)  # Set to frame number 100
ret, frame = cap.read()
cap.release()

if ret:
    
    cv2.line(frame, start_point, end_point, (255, 120, 255), 10)
    
    TEXT = "COUNTING LINE"
    cv2.putText(frame, TEXT, (1510, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 120, 255), 10)
    
    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

### **Fruits Counting Logic**

This is the main part of the application. We define a `process_video` function that handles everything: loading the model, reading the video, running the tracker on each frame, and implementing the counting logic. It tracks the center point of each detected fruit and uses a line intersection function to check if a fruit's path has crossed our predefined counter line. If it has, and the fruit hasn't been counted before, the total count is incremented. The final video is saved with visual overlays showing the masks, tracking IDs, the counting line, and the live count.

In [ ]:
from datetime import datetime

# =============================================================================
# GLOBAL VARIABLES
# =============================================================================
product_counter = 0
perform_segmentation = False  # Set to True to enable segmentation visualization
counting_line = (line)  # line coordinates
video_path = "assests/2.mp4"
output_video_path = "output5.mp4"
model_path = "./runs/segment/train/weights/best.pt"  # Trained segmentation model
model_confidence = 0.9  # Confidence threshold for YOLO model

# =============================================================================
# FUNCTIONS
# =============================================================================
def load_yolo_model(model_path):
    """Load YOLO model"""
    global model
    try:
        print(f"Loading YOLO segmentation model: {model_path}")
        model = YOLO(model_path)
        print("YOLO model loaded successfully")
        return True
    except Exception as e:
        print(f"Error loading YOLO model: {e}")
        return False


def line_intersection(p1, p2, p3, p4):
    """Check if line p1-p2 intersects with line p3-p4"""
    def ccw(A, B, C):
        return (C[1] - A[1]) * (B[0] - A[0]) > (B[1] - A[1]) * (C[0] - A[0])
    return ccw(p1, p3, p4) != ccw(p2, p3, p4) and ccw(p1, p2, p3) != ccw(p1, p2, p4)


def check_line_crossing(prev_pos, curr_pos, obj_id):
    """Check if object crosses the counting line"""
    global product_counter, counted_objects
    if line_intersection(prev_pos, curr_pos, counting_line[0], counting_line[1]):
        if obj_id not in counted_objects:  # Only count once per object
            counted_objects.add(obj_id)
            product_counter += 1
            print(f"🎯 Object {obj_id} crossed the line! Total count: {product_counter}")


def process_video():
    """Main function to process video"""
    global product_counter, counted_objects
    
    print(f"Starting Product Counter")
    print(f"Input: {video_path}")
    print(f"Output: {output_video_path}")
    print(f"Model: {model_path}")
    print("="*50)

    # Load model
    if not load_yolo_model(model_path):
        return False

    # Open input video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Cannot open video: {video_path}")
        return False

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Video: {width}x{height} @ {fps}fps, {total_frames} frames")

    # Create output video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Reset counter and tracking
    product_counter = 0
    counted_objects = set()  # Track which objects have been counted
    track_history = {}  # {id: (prev_center)}

    frame_count = 0
    start_time = datetime.now()

    print("Processing...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Run YOLO segmentation + tracking
        results = model.track(frame, conf=model_confidence, tracker="botsort.yaml", persist=True, verbose=False)

        if results and results[0].boxes.id is not None:
            # Draw segmentation masks with color coding
            if results[0].masks is not None:
                masks = results[0].masks.xy  # list of polygons
                boxes = results[0].boxes
                
                if perform_segmentation == True:
                    for i, seg in enumerate(masks):
                        if i < len(boxes):
                            obj_id = int(boxes[i].id[0].cpu().numpy())
                            
                            # Color based on counting status
                            if obj_id in counted_objects:
                                # Yellow translucent for counted objects
                                color = (0, 255, 255)  # BGR format: Yellow
                                fill_color = (0, 255, 255, 100)  # Yellow with alpha
                            else:
                                # Purple translucent for uncounted objects
                                color = (255, 0, 255)  # BGR format: Purple/Magenta
                                fill_color = (255, 0, 255, 100)  # Purple with alpha
                            
                            pts = np.array(seg, dtype=np.int32)
                            
                            # Create overlay for translucent fill
                            overlay = frame.copy()
                            cv2.fillPoly(overlay, [pts], color)
                            cv2.addWeighted(overlay, 0.3, frame, 0.7, 0, frame)
                            
                            # Draw outline
                            cv2.polylines(frame, [pts], True, color, 2)

            # Draw tracked objects
            for box in results[0].boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                obj_id = int(box.id[0].cpu().numpy())
                cls_id = int(box.cls[0].cpu().numpy())
                conf = float(box.conf[0].cpu().numpy())

                # Object center
                center_x = int((x1 + x2) / 2)
                center_y = int((y1 + y2) / 2)
                center = (center_x, center_y)

                # Check line crossing
                if obj_id in track_history:
                    prev_center = track_history[obj_id]
                    check_line_crossing(prev_center, center, obj_id)
                track_history[obj_id] = center

                # Draw bounding box with color coding
                box_color = (0, 255, 255) if obj_id in counted_objects else (255, 0, 255)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), box_color, 2)
                cv2.putText(frame, f"ID:{obj_id}", (int(x1), int(y1) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                cv2.circle(frame, center, 5, (0, 0, 255), -1)

        # Draw counting line
        cv2.line(frame, counting_line[0], counting_line[1], (0, 255, 0), 10)
        cv2.putText(frame, "COUNTING LINE",
                    ((counting_line[0][0] + counting_line[1][0]) // 2 + 20,
                     (counting_line[0][1] + counting_line[1][1]) // 2),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 10)

        # Draw counter
        cv2.rectangle(frame, (10, 10), (200, 60), (0, 0, 0), -1)
        cv2.putText(frame, f"COUNT: {product_counter}", (20, 45),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

        out.write(frame)

        # Show progress every 10%
        if total_frames > 0 and frame_count % max(1, total_frames // 10) == 0:
            progress = (frame_count / total_frames) * 100
            print(f"📈 {progress:.0f}% - Frame {frame_count}/{total_frames} - Count: {product_counter}")

    # Cleanup
    cap.release()
    out.release()

    end_time = datetime.now()
    processing_time = end_time - start_time

    # Results
    print("="*50)
    print("Processing completed!")
    print(f"Total count: {product_counter}")
    print(f"Processing time: {processing_time}")
    print(f"Output saved: {output_video_path}")

In [ ]:
process_video()

---

## 👨‍💻 About Labellerr's Hands-On Learning in Computer Vision

Thank you for exploring this **Labellerr Hands-On Computer Vision Cookbook**! We hope this notebook helped you learn, prototype, and accelerate your vision projects.  
Labellerr provides ready-to-run Jupyter/Colab notebooks for the latest models and real-world use cases in computer vision, AI agents, and data annotation.

---
## 🧑‍🔬 Check Our Popular Youtube Videos

Whether you're a beginner or a practitioner, our hands-on training videos are perfect for learning custom model building, computer vision techniques, and applied AI:

- [How to Fine-Tune YOLO on Custom Dataset](https://www.youtube.com/watch?v=pBLWOe01QXU)  
  Step-by-step guide to fine-tuning YOLO for real-world use—environment setup, annotation, training, validation, and inference.
- [Build a Real-Time Intrusion Detection System with YOLO](https://www.youtube.com/watch?v=kwQeokYDVcE)  
  Create an AI-powered system to detect intruders in real time using YOLO and computer vision.
- [Finding Athlete Speed Using YOLO](https://www.youtube.com/watch?v=txW0CQe_pw0)  
  Estimate real-time speed of athletes for sports analytics.
- [Object Counting Using AI](https://www.youtube.com/watch?v=smsjBBQcIUQ)  
  Learn dataset curation, annotation, and training for robust object counting AI applications.
---

## 🎦 Popular Labellerr YouTube Videos

Level up your skills and see video walkthroughs of these tools and notebooks on the  
[Labellerr YouTube Channel](https://www.youtube.com/@Labellerr/videos):

- [How I Fixed My Biggest Annotation Nightmare with Labellerr](https://www.youtube.com/watch?v=hlcFdiuz_HI) – Solving complex annotation for ML engineers.
- [Explore Your Dataset with Labellerr's AI](https://www.youtube.com/watch?v=LdbRXYWVyN0) – Auto-tagging, object counting, image descriptions, and dataset exploration.
- [Boost AI Image Annotation 10X with Labellerr's CLIP Mode](https://www.youtube.com/watch?v=pY_o4EvYMz8) – Refine annotations with precision using CLIP mode.
- [Boost Data Annotation Accuracy and Efficiency with Active Learning](https://www.youtube.com/watch?v=lAYu-ewIhTE) – Speed up your annotation workflow using Active Learning.

> 👉 **Subscribe** for Labellerr's deep learning, annotation, and AI tutorials, or watch videos directly alongside notebooks!

---

## 🤝 Stay Connected

- **Website:** [https://www.labellerr.com/](https://www.labellerr.com/)
- **Blog:** [https://www.labellerr.com/blog/](https://www.labellerr.com/blog/)
- **GitHub:** [Labellerr/Hands-On-Learning-in-Computer-Vision](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)
- **LinkedIn:** [Labellerr](https://in.linkedin.com/company/labellerr)
- **Twitter/X:** [@Labellerr1](https://x.com/Labellerr1)

*Happy learning and building with Labellerr!*
